In [ ]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
spark

In [ ]:
df_table = spark.read.csv('vca.csv', header=True, inferSchema=True)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
table2018 = df_table.where('2018_isplanmember=true') \
                                        .withColumn('Year', F.lit(2018)) \
                                        .select(['Year','speciesname', '2018_planname', 'patientid', '2018_revenue']) \
                                        .withColumnRenamed('2018_planname', 'PlanName') \
                                        .withColumnRenamed('2018_revenue', 'Revenue')
table2019 =   df_table.where('2018_isplanmember=true and 2019_isplanmember=true') \
                                        .withColumn('Year', F.lit(2019)) \
                                        .select(['Year','speciesname', '2018_planname', 'patientid', '2019_revenue']) \
                                        .withColumnRenamed('2018_planname', 'PlanName') \
                                        .withColumnRenamed('2019_revenue', 'Revenue') 
table2020 =   df_table.where('2018_isplanmember=true and 2019_isplanmember=true and 2020_isplanmember=true') \
                                        .withColumn('Year', F.lit(2020)) \
                                        .select(['Year','speciesname', '2018_planname', 'patientid', '2020_revenue']) \
                                        .withColumnRenamed('2018_planname', 'PlanName') \
                                        .withColumnRenamed('2020_revenue', 'Revenue') 
table2021 =   df_table.where('2018_isplanmember=true and 2019_isplanmember=true and 2020_isplanmember=true and 2021_isplanmember = true') \
                                        .withColumn('Year', F.lit(2021)) \
                                        .select(['Year','speciesname', '2018_planname', 'patientid', '2021_revenue']) \
                                        .withColumnRenamed('2018_planname', 'PlanName') \
                                        .withColumnRenamed('2021_revenue', 'Revenue') 
Sampletable = table2018.unionAll(table2019).unionAll(table2020).unionAll(table2021)
Sampletable.toPandas().head(2)

In [ ]:
Sampletable.write.mode("overwrite").csv("Sampletable")

In [ ]:
Sampletable.write.mode("overwrite").saveAsTable("Sampletable")

In [ ]:
spark.sql('''select * from `Sampletable`''')

In [ ]:
#Sampletable.first()
#Sampletable.filter('Year == 2018').show(5)
#Sampletable.filter('Year == 2018').limit(5).display()
#Sampletable.head(5)
#Sampletable.limit(5).display()
#Sampletable.columns
#Sampletable.count()
#Sampletable.describe().display()
#Sampletable.count()
#Sampletable.select('patientid').count()
#Sampletable.drop_duplicates(['patientid']).count()
#Sampletable.select('patientid').distinct().count()
#Sampletable.groupby('Year').sum('Revenue').display()
#Sampletable.groupby('Year').agg(F.sum(Sampletable.Revenue).alias('Total_Revenue'), F.countDistinct('patientid').alias('Patient')).display()
#Sampletable.groupby('Year').agg(sum(Sampletable.Revenue).alias('Total_Revenue'), countDistinct('patientid').alias('Patient')).display()
#Sampletable.groupby('Year').agg(sum(Sampletable.Revenue).alias('Total_Revenue'), countDistinct('patientid').alias('Patient')).withColumn('test', F.lit("test")).display()
#Sampletable.select(countDistinct('patientid')).display()
#Sampletable.orderBy('Year', ascending=False).show(5)
#Sampletable.sort('Year', ascending=False).show(5)

In [ ]:
df_table.filter('2018_isplanmember = "true"').selectExpr('''
count(distinct patientid)
''').show()

In [ ]:
df_table.withColumn('Year', F.lit(2018)) \
.filter('2018_isplanmember = "true"')   \
.groupBy('Year').agg((F.expr('''
count(distinct patientid)''')*100/F.expr('''
count(distinct case when 2018_isplanmember = "true" then patientid end)
''')).alias('SurvivalRate')).show()

In [ ]:
df_table.selectExpr('''
2018 as Year
''').show(2)